For updating: <br>
In this guided project, we'll work with stock market data downloaded from Yahoo Finance using the yahoo_finance Python package. This data consists of the daily stock prices from 2007-1-1 to 2017-04-17 for several hundred stock symbols traded on the NASDAQ stock exchange, stored in the prices folder. We used the download_data.py script in the same folder as the Jupyter notebook to download all of the stock price data. 

In [1]:
import pandas as pd
import os

from IPython.display import display, Markdown

In [2]:
# Iniialize dictionary
prices_df_dict = {}

In [3]:
#  fn means filename

for fn in os.listdir("prices"):
    key = fn.split(".csv")[0]
    
    df = pd.read_csv(os.path.join("prices", fn))
    prices_df_dict[key] = df


Calculate the average values for each stock

In [4]:
stock_mean_dict = {}
for stock_name in prices_df_dict.keys():
    stock_mean_dict[stock_name] = prices_df_dict[stock_name].mean()
    

In [5]:
# convert the dictionary into a dataframe
# get the target analysis
# optimize the process using multiprocessing (apply what you've learned to make it stick)

In [6]:
stock_mean_df = pd.DataFrame(stock_mean_dict).T
stock_mean_df.head()

,close,open,high,low,volume
dgica,14.986583,14.986066,15.206610,14.742745,27233.783784
bdge,24.120351,24.118892,24.400737,23.828386,20277.953668
cvco,53.365436,53.362023,54.303954,52.433988,40272.393822
blkb,33.755378,33.719023,34.185849,33.245826,242526.563707
bbox,25.997579,26.016255,26.463915,25.532606,97324.401544


In [7]:
def color_text(color, text):
    return f'<span style=\"color:{color}\">{text}</span>'

In [8]:
stock_idx_max = stock_mean_df["close"].idxmax()
stock_idx_min = stock_mean_df["close"].idxmin()

rounded_price_max = round(stock_mean_df.loc[stock_idx_max]['close'], 2)
rounded_price_min = round(stock_mean_df.loc[stock_idx_min]['close'], 2)

stock_message_1 = f"**{stock_idx_max}** stock has the maximum average closing price of {color_text('green', rounded_price_max)}"
stock_message_2 = f"**{stock_idx_min}** stock is on the other end with average closing price of {color_text('red', rounded_price_min)}"

display(Markdown("Comparing all stock closing prices:"))
display(Markdown(stock_message_1))
display(Markdown(stock_message_2))

Comparing all stock closing prices:

**amzn** stock has the maximum average closing price of <span style="color:green">275.13</span>

**blfs** stock is on the other end with average closing price of <span style="color:red">0.81</span>

In [9]:
# vol_stk_pair_dict = {}
# for stock in prices_df_dict.keys():
#     df = prices_df_dict[stock]
#     df_grouped = df.groupby(by=["date"]).sum()["volume"]
#     df_grouped_vol = pd.DataFrame(df_grouped)
#     df_grouped_vol[f"vol-stk-{stock}"] = [(volume, stock) for volume in df_grouped_vol["volume"] ]
#     df_grouped_vol = df_grouped_vol.drop("volume", axis=1)
#     vol_stk_pair_dict[stock] = df_grouped_vol
    
    
# merged_df = pd.DataFrame()
# for stock in vol_stk_pair_dict.keys():
#     current_df = vol_stk_pair_dict[stock]
#     merged_df= merged_df.merge(current_df, left_index=True, right_index=True, how="outer")

In [10]:
def create_volume_stock_pair_dict(prices_df_dict):
    """
    Parameters: 
        prices_df_dict -- dictionary containing stock symbol as key and the dataframed values for that stock as values
    Output:
        vol_stk_pair_dict -- a dictionary with stock symbol as key and the df containing data with the format (volume, stock)
    """
    print("Creating volumes dictionary", end="::")
    vol_stk_pair_dict = {}
    stock_counter = 0
    for stock in prices_df_dict.keys():
        df = prices_df_dict[stock]
        df_grouped = df.groupby(by=["date"]).sum()["volume"]
        df_grouped_vol = pd.DataFrame(df_grouped)
        
        df_grouped_vol[f"vol-stk-{stock}"] = [(volume, stock) if volume !=0 else None for volume in df_grouped_vol["volume"]]
        
        df_grouped_vol = df_grouped_vol.drop("volume", axis=1)
        vol_stk_pair_dict[stock] = df_grouped_vol
        
        if stock_counter%100==0:
            print(".", end="")
        stock_counter += 1
        
    return vol_stk_pair_dict


def merge_dataframes(vol_stk_pair_dict):
    """
    Parameters:
    vol_stk_pair_dict -- a dictionary with stock symbol as key and the df containing data with the format (volume, stock)
    
    Output:
    merged_df -- All the dataframes merged by index using outer join
    """
    merged_df = pd.DataFrame()
    stock_counter = 0
    print("Merging all dataframes in volumes dictionary into a single dataframe", end="::")
    for stock in vol_stk_pair_dict.keys():
        current_df = vol_stk_pair_dict[stock]
        merged_df = merged_df.merge(current_df, left_index=True, right_index=True, how="outer")
        
        if stock_counter%100==0:
            print(".", end="")
        stock_counter += 1
        
    return merged_df

In [11]:
def get_volume(df, date, stock_symbol):
    volume_series = df[df["date"]==date]["volume"]
    vol_stock_pair_list = []
    for vol in volume_series:
        vol_stock_pair_list.append((vol, stock_symbol))
    return vol_stock_pair_list

def get_day_trades(dict_name):
    '''
    Do NOT use. Too much nested for-loops. =p
    Output: Returns a dictionary
    '''
    dict_days_trades = {}
    for stock in dict_name.keys():
        df = dict_name[stock]
        for date in df["date"].unique():
            dict_days_trades[date] = get_volume(df, date, stock)

    return dict_days_trades

In [12]:
vol_stk_pair_dict = create_volume_stock_pair_dict(prices_df_dict)

merged_df = merge_dataframes(vol_stk_pair_dict)

merged_df.head()

............

,vol-stk-dgica,vol-stk-bdge,vol-stk-cvco,vol-stk-blkb,vol-stk-bbox,vol-stk-ffbc,vol-stk-fbiz,vol-stk-ffic,vol-stk-bdsi,vol-stk-amgn,...,vol-stk-anat,vol-stk-bbh,vol-stk-aiq,vol-stk-colb,vol-stk-bbsi,vol-stk-banr,vol-stk-anss,vol-stk-dynt,vol-stk-eng,vol-stk-flic
date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,"(30100, dgica)","(100, bdge)","(36500, cvco)","(365800, blkb)","(108200, bbox)","(192600, ffbc)","(400, fbiz)","(46300, ffic)","(29500, bdsi)","(12908400, amgn)",...,"(7600, anat)","(1002900, bbh)","(39400, aiq)","(108700, colb)","(76100, bbsi)","(5900, banr)","(1159800, anss)","(1100, dynt)","(162200, eng)","(57900, flic)"
2007-01-04,"(16800, dgica)","(1300, bdge)","(18500, cvco)","(287500, blkb)","(91800, bbox)","(177600, ffbc)",None,"(48900, ffic)","(23300, bdsi)","(16000900, amgn)",...,"(1600, anat)","(1638900, bbh)","(44900, aiq)","(186000, colb)","(27500, bbsi)","(7000, banr)","(1238200, anss)","(2700, dynt)","(127400, eng)","(5700, flic)"
2007-01-05,"(27700, dgica)","(2200, bdge)","(30400, cvco)","(251200, blkb)","(125100, bbox)","(165200, ffbc)",None,"(52600, ffic)","(73400, bdsi)","(10462000, amgn)",...,"(7100, anat)","(699900, bbh)","(38100, aiq)","(141700, colb)","(32400, bbsi)","(5000, banr)","(851600, anss)","(3500, dynt)","(179900, eng)","(600, flic)"
2007-01-08,"(30300, dgica)","(4700, bdge)","(29500, cvco)","(180600, blkb)","(91000, bbox)","(140400, ffbc)","(3200, fbiz)","(81900, ffic)","(10600, bdsi)","(6747100, amgn)",...,"(1200, anat)","(524400, bbh)","(99200, aiq)","(50000, colb)","(31700, bbsi)","(2100, banr)","(1249800, anss)","(300, dynt)","(123500, eng)",None
2007-01-09,"(37800, dgica)","(3000, bdge)","(22800, cvco)","(529900, blkb)","(203600, bbox)","(90700, ffbc)","(13600, fbiz)","(105100, ffic)","(24200, bdsi)","(7165200, amgn)",...,"(2200, anat)","(1059000, bbh)","(29900, aiq)","(49100, colb)","(22200, bbsi)","(3600, banr)","(408400, anss)","(500, dynt)","(173200, eng)","(6900, flic)"


In [13]:
def rename_columns_volume_stk_pair(df):
    new_col_names = []
    for col_name in df.columns:
        if "vol-stk-" in col_name:
            new_col = col_name.split("vol-stk-")[1]
        else:
            new_col = col_name
        new_col_names.append(new_col)
        
    df.columns = new_col_names

Create a column containing an array of all (volume, stock) tuple for each day.

In [14]:
# Rename the columns to the stock symbol
rename_columns_volume_stk_pair(merged_df)

# Combine all the volume-stock symbol pair data for each day into a single column
merged_df["combined_volumes"] = merged_df.apply(lambda row: row.dropna().to_list(), axis= 1 )
merged_df["combined_volumes"].head()

date
2007-01-03    [(30100, dgica), (100, bdge), (36500, cvco), (...
2007-01-04    [(16800, dgica), (1300, bdge), (18500, cvco), ...
2007-01-05    [(27700, dgica), (2200, bdge), (30400, cvco), ...
2007-01-08    [(30300, dgica), (4700, bdge), (29500, cvco), ...
2007-01-09    [(37800, dgica), (3000, bdge), (22800, cvco), ...
Name: combined_volumes, dtype: object

Inspect the traded volume and stocks array for a specific date

In [15]:
merged_df["combined_volumes"].loc["2007-01-03"][:10]

[(30100, 'dgica'),
 (100, 'bdge'),
 (36500, 'cvco'),
 (365800, 'blkb'),
 (108200, 'bbox'),
 (192600, 'ffbc'),
 (400, 'fbiz'),
 (46300, 'ffic'),
 (29500, 'bdsi'),
 (12908400, 'amgn')]

In [16]:
# Create a function that will return tuple with the maximum volume traded for each day
def max_first_element(tuples_array):
    if tuples_array:
        return max(tuples_array, key=lambda x: x[0])
    else:
        return None

Find the stock with the largest volume traded for each day

In [17]:
merged_df['hot'] = merged_df["combined_volumes"].apply(max_first_element)

In [18]:
merged_df['hot'].head()

date
2007-01-03    (309579900, aapl)
2007-01-04    (211815100, aapl)
2007-01-05    (208685400, aapl)
2007-01-08    (199276700, aapl)
2007-01-09    (837324600, aapl)
Name: hot, dtype: object